In [1]:
import numpy as np
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -qq langdetect
from langdetect import detect
import string
!pip install -qq contractions
!pip install -qq demoji
import demoji
demoji.download_codes()
!pip install -qq datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.4 MB/s eta 0:00:00


<ipython-input-2-28a5514cb77a>:7: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00


In [3]:
!pip install -qq inltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.1/354.1 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 7.1 MB/s eta 0:00:00


In [4]:
header_names = ['label', 'text']
df_train = pd.read_csv("/content/drive/MyDrive/Capstone/Dataset/tamil_offensive_full.csv", sep='\t',names=header_names)
df_train = df_train.dropna(how='all')
df_train = pd.DataFrame(df_train)
df_train

,label,text
0,Offensive_Untargetede,Lighta adanga maaru bgm maathiri theriyuthu
1,Offensive_Untargetede,Sema...last dialogue awesome....sanda pottu ed...
2,Offensive_Targeted_Insult_Other,Nice But pesura dialogue onnum puriyala romba...
3,Not_offensive,BGM asusual therikka vitaru Sam cs
4,Not_offensive,Entha Diwali Kaithi Diwali
...,...,...
43914,Not_offensive,கவுண்டர் சமுதாய மக்கள் சார்பாக வெற்றி பெற வாழ்...
43915,Offensive_Targeted_Insult_Group,Comment la periya pudungi madiri jaadi name ah...
43916,Not_offensive,திரைப்படம் பொங்கலுக்கு ரிலீசானால் நன்றாக இருக்...
43917,Not_offensive,Sathi verundri valara itha Padam oru eduthukattu


In [6]:
label_mapping = {
    'Not_offensive': 0,
    'Offensive_Untargetede': 1,
    'Offensive_Targeted_Insult_Individual': 1,
    'Offensive_Targeted_Insult_Group': 1,
    'Offensive_Targeted_Insult_Other': 1,
    'not-Tamil': 2,
}
df_train['label'].replace(label_mapping, inplace=True)
df_train = df_train[df_train['label'] != 2]
print(df_train.info())
label_counts = df_train['label'].value_counts()
print(label_counts)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42133 entries, 0 to 43918
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   42133 non-null  int64 
 1   text    42133 non-null  object
dtypes: int64(1), object(1)
memory usage: 987.5+ KB
None
0    31808
1    10325
Name: label, dtype: int64


In [7]:
df_train.head(20)

,label,text
0,1,Lighta adanga maaru bgm maathiri theriyuthu
1,1,Sema...last dialogue awesome....sanda pottu ed...
2,1,Nice But pesura dialogue onnum puriyala romba...
3,0,BGM asusual therikka vitaru Sam cs
4,0,Entha Diwali Kaithi Diwali
5,1,"Trailer nalla dhan iruku, theva illama en #BIG..."
6,1,Bigil kuda realsee panatha da unga padam odathu
7,0,Waiting for lokesh kanakaraj Thalapathy 64 Th...
8,1,Indha padathoda trailer eh indha level la iruk...
9,0,Screenplay vera level laaa iruku.... Waiting t...


In [8]:
import sys
sys.path.append("/content/drive/MyDrive/Capstone/Translator/")
from google_trans_new import google_translator

In [9]:
translator = google_translator()
for index, row in df_train.iterrows(): # Check if emoji is contained in text
    text = str(row['text'])
    found = demoji.findall(text)
    if found:
        # There is an emoji now detect language
        if detect(text) == 'ta': # Text is Tamil
            for key, value in found.items():
                emojiEng = value
                # Translate to Tamil
                transText = translator.translate(emojiEng, lang_tgt='ta', lang_src='en')
                text = text.replace(key, transText + ' ')
        else: # Text is english
            for key, value in found.items():
                text = text.replace(key, value + ' ')

        df_train.at[index, 'text'] = text

In [10]:
df_train

,label,text
0,1,Lighta adanga maaru bgm maathiri theriyuthu
1,1,Sema...last dialogue awesome....sanda pottu ed...
2,1,Nice But pesura dialogue onnum puriyala romba...
3,0,BGM asusual therikka vitaru Sam cs
4,0,Entha Diwali Kaithi Diwali
...,...,...
43914,0,கவுண்டர் சமுதாய மக்கள் சார்பாக வெற்றி பெற வாழ்...
43915,1,Comment la periya pudungi madiri jaadi name ah...
43916,0,திரைப்படம் பொங்கலுக்கு ரிலீசானால் நன்றாக இருக்...
43917,0,Sathi verundri valara itha Padam oru eduthukattu


In [11]:
for index, row in df_train.iterrows():
    found = demoji.findall(row['text'])
    if found:
        print('Found exception')

In [12]:
import contractions
# contracted text
for index, row in df_train.iterrows():# creating an empty list
    expanded_words = []
    for word in str(row['text']).split():# using contractions.fix to expand the shotened words
        expanded_words.append(contractions.fix(word))

    expanded_text = ' '.join(expanded_words)
    df_train.at[index, 'text'] = expanded_text

for index, row in df_train.iterrows():
    Sentence = str(row['text'])
    for i in Sentence:
        # checking whether the char is punctuation.
        if i in string.punctuation:
            Sentence = Sentence.replace(i, '')
    df_train.at[index, 'text'] = Sentence.lower()

In [13]:
df_train.head(20)

,label,text
0,1,lighta adanga maaru bgm maathiri theriyuthu
1,1,semalast dialogue awesomesanda pottu edukku sa...
2,1,nice but pesura dialogue onnum puriyala romba ...
3,0,bgm asusual therikka vitaru sam cs
4,0,entha diwali kaithi diwali
5,1,trailer nalla dhan iruku theva illama en bigil...
6,1,bigil kuda realsee panatha da unga padam odathu
7,0,waiting for lokesh kanakaraj thalapathy 64 tha...
8,1,indha padathoda trailer eh indha level la iruk...
9,0,screenplay vera level laaa iruku waiting to saw


In [14]:
!pip install -qq advertools
import advertools as adv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.8/287.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 8.4 MB/s eta 0:00:00


In [15]:
stopwords = adv.stopwords['tamil']
data=[]
for index, row in df_train.iterrows():
    sent = ''
    line = str(row['text'])
    for i in line.split():
        if i not in stopwords:
            sent += i + " "
    data.append({'text': sent, 'label': str(row['label'])})


new_df_train = pd.DataFrame(data)
new_df_train

,text,label
0,lighta adanga maaru bgm maathiri theriyuthu,1
1,semalast dialogue awesomesanda pottu edukku sa...,1
2,nice but pesura dialogue onnum puriyala romba ...,1
3,bgm asusual therikka vitaru sam cs,0
4,entha diwali kaithi diwali,0
...,...,...
42128,கவுண்டர் சமுதாய மக்கள் சார்பாக வெற்றி பெற வாழ்...,0
42129,comment la periya pudungi madiri jaadi name ah...,1
42130,திரைப்படம் பொங்கலுக்கு ரிலீசானால் நன்றாக,0
42131,sathi verundri valara itha padam oru eduthukattu,0


In [16]:
import re

# Function to remove URLs from text
def remove_urls(text):
    return re.split(r'https?:\/\/.*', str(text))[0]

# Apply the function to the 'text' column in new_df_train
new_df_train['text'] = new_df_train['text'].apply(remove_urls)

new_df_train.head(10)


,text,label
0,lighta adanga maaru bgm maathiri theriyuthu,1
1,semalast dialogue awesomesanda pottu edukku sa...,1
2,nice but pesura dialogue onnum puriyala romba ...,1
3,bgm asusual therikka vitaru sam cs,0
4,entha diwali kaithi diwali,0
5,trailer nalla dhan iruku theva illama en bigil...,1
6,bigil kuda realsee panatha da unga padam odathu,1
7,waiting for lokesh kanakaraj thalapathy 64 tha...,0
8,indha padathoda trailer eh indha level la iruk...,1
9,screenplay vera level laaa iruku waiting to saw,0


In [17]:
new_df_train['text'].isnull().sum()

0

In [23]:
!pip install -qq indic-nlp-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 14.2 MB/s eta 0:00:00


In [24]:
from indicnlp import common
from indicnlp import loader
from indicnlp.script import indic_scripts
from indicnlp.tokenize import indic_tokenize
from indicnlp.morph import unsupervised_morph

In [19]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 26.26 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Updating files: 100% (28/28), done.


In [25]:
INDIC_NLP_RESOURCES = r"/content/indic_nlp_resources"
common.set_resources_path(INDIC_NLP_RESOURCES)
loader.load()
analyzer = unsupervised_morph.UnsupervisedMorphAnalyzer('ta')

INFO:morfessor.io:Loading model from '/content/indic_nlp_resources/morph/morfessor/ta.model'...
INFO:morfessor.io:Reading segmentations from '/content/indic_nlp_resources/morph/morfessor/ta.model'...
INFO:morfessor.io:Detected utf-8 encoding
INFO:morfessor.io:Done.
INFO:morfessor.io:/content/indic_nlp_resources/morph/morfessor/ta.model was read as a segmentation


In [26]:
from langdetect import detect, LangDetectException

In [27]:
from tqdm import tqdm

def stem_tamil_text(text):
    tokens = text.split()
    stemmed_tokens = []
    for token in tokens:
        analyzed_tokens = analyzer.morph_analyze_document([token])
        if analyzed_tokens:
            stemmed_tokens.append(analyzed_tokens[0])
        else:
            stemmed_tokens.append(token)  # If stemming fails, keep the original token
    return ' '.join(stemmed_tokens)

# Assuming new_df_train is your DataFrame containing text data
# Add tqdm to the loop to track progress
for index, row in tqdm(new_df_train.iterrows(), total=len(new_df_train)):
    text = str(row['text'])

    try:
        detected_lang = detect(text)
    except LangDetectException as e:
        # Handle the exception, for example by setting the detected_lang to None
        detected_lang = None

    if detected_lang == 'ta':
        stemmed_text = stem_tamil_text(text)
        new_df_train.at[index, 'text'] = stemmed_text


print(new_df_train.head(20))


100%|██████████| 42133/42133 [10:13<00:00, 68.65it/s] 

                                                 text label
0        lighta adanga maaru bgm maathiri theriyuthu      1
1   semalast dialogue awesomesanda pottu edukku sa...     1
2   nice but pesura dialogue onnum puriyala romba ...     1
3                 bgm asusual therikka vitaru sam cs      0
4                         entha diwali kaithi diwali      0
5   trailer nalla dhan iruku theva illama en bigil...     1
6    bigil kuda realsee panatha da unga padam odathu      1
7   waiting for lokesh kanakaraj thalapathy 64 tha...     0
8   indha padathoda trailer eh indha level la iruk...     1
9    screenplay vera level laaa iruku waiting to saw      0
10  thalapathy 64 director lokesh kanagaraj like p...     0
11  commercial kuppai igil ku thriller story kaith...     1
12  kaithi trailer ah paarthutu thalapathy 64 ku v...     1
13              dislike potavan ellam anil fans pola      1
14           next bigil movie ku waiting pantravanga      0
15  idhuku dislike pota naainga ratham k

In [28]:
new_df_train.to_csv('tamil_dataset_2_train.csv', index=False)

In [29]:
new_df_train

,text,label
0,lighta adanga maaru bgm maathiri theriyuthu,1
1,semalast dialogue awesomesanda pottu edukku sa...,1
2,nice but pesura dialogue onnum puriyala romba ...,1
3,bgm asusual therikka vitaru sam cs,0
4,entha diwali kaithi diwali,0
...,...,...
42128,கவுண்டர் சமுதாய மக்கள் சார்பாக வெற்றி பெற வாழ்...,0
42129,comment la periya pudungi madiri jaadi name ah...,1
42130,திரைப்பட பொங்கலுக்கு ரிலீசான நன்றாக,0
42131,sathi verundri valara itha padam oru eduthukattu,0
